In [ ]:
-- TODO set up dummy network
-- TODO set up mask-making code
-- TODO set up criterion code 
-- TODO test noopmask+criterion against regular critierion
-- TODO test real mask with dummy annealing
-- TODO test real thing


In [ ]:
require 'nn';


net = nn.Sequential()
net:add(nn.SpatialConvolution(1, 6, 5, 5)) -- 1 input image channel, 6 output channels, 5x5 convolution kernel
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.SpatialMaxPooling(2,2,2,2))     -- A max-pooling operation that looks at 2x2 windows and finds the max.
net:add(nn.SpatialConvolution(6, 16, 5, 5))
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.SpatialMaxPooling(2,2,2,2))
net:add(nn.View(16*5*5))                    -- reshapes from a 3D tensor of 16x5x5 into 1D tensor of 16*5*5
net:add(nn.Linear(16*5*5, 120))             -- fully connected layer (matrix multiplication between input and weights)
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.Linear(120, 84))
net:add(nn.ReLU())                       -- non-linearity 
net:add(nn.Linear(84, 10))                   -- 10 is the number of outputs of the network (in this case, 10 digits)
net:add(nn.LogSoftMax())                     -- converts the output to a log-probability. Useful for classification problems

In [ ]:
input = torch.rand(1,32,32) -- pass a random tensor as input to the network
output = net:forward(input)
--print(input)
print(output:exp())

In [ ]:
criterion = nn.ClassNLLCriterion() -- a negative log-likelihood criterion for multi-class classification
error = criterion:forward(output, 3) -- let's say the groundtruth was class number: 3
gradients = criterion:backward(output, 4)
gradInput = net:backward(input, gradients)
print(error)
print(criterion)
print(gradients)

In [ ]:
asdf = torch.ones(10)
blah = torch.zeros(10)
blah[7] = 1
qwer = torch.cmul(blah, asdf)
print(blah)
print(asdf)
print(qwer)

In [1]:
-- TODO I don't think using cmul is correct. You have to make sure you multiply each result forward by the mask.
-- TODO test this out on some REAL data from provider.

-- TODO there are special functions for masks, e.g. maskedCopy, maskedFill, try those...

require 'nn';

do

-- Construct a new one of these every mini batch, because you need a new unlabledMask for each one.
-- This assumes unlabeled have labeled of 0.
local PseudoLabelCriterion, Criterion = torch.class('nn.PseudoLabelCriterion', 'nn.Criterion')

-- crit: Criterion object
-- unlabeledMask: a 1 x n tensor, where n is the mini-batch size.
--                It has zeros for labeled data, and ones for unlabeled data.
-- kAnneal: a scalar we use to weight the unlabled components of the gradient of the loss 
function PseudoLabelCriterion:__init(crit, unlabeledMask, kAnneal)
   Criterion.__init(self)
   self.crit = crit
   self.unlabeledMask = unlabeledMask
   -- The unlabled mask, but the ones are replaced with kAnneal, and zeros are replaced with ones.
        -- TODO do I have to cuda this?
   if (type(unlabeledMask) == 'number') then
      self.annealedUnlabeledMask = unlabeledMask*(kAnneal-1) + 1
   else
      -- putting this into a diagonal matrix allows you to multiply rows easily
      self.annealedUnlabeledMask = torch.diag(unlabeledMask*(kAnneal-1) + torch.ones(length(unlabeledMask)))
   end
end


-- returns/sets the error
-- NOTE: modifies target. we assume target has zeros for the unlabeled data.
-- input must be n x k matrix, where n is the mini batch size and k is the number of classes.
-- target must be a 1 x n matrix with the correct labels
function PseudoLabelCriterion:updateOutput(input, target)
   -- squeeze gets rid of extra dimensions.
   input = input:squeeze()
   if type(target) == 'number' then -- single dimension case
      if unlabeledMask == 1 then
         _, predictedLabel = torch.max(input, 1)
         target = predictedLabel
      end
      self.crit:updateOutput(input, target)
      self.crit.output = self.crit.output * self.annealedUnlabeledMask
   else
      target = target:squeeze()
      -- Get the max indexes of the target, which gets you the predicted labels      
      _, predictedLabels = torch.max(input, 2) -- TODO TODO is this the right dimension???? maybe 1?
      -- keep the predicted labels of the unlabeled data.
      -- This assumes the unlabeled data has a target of zero.   
      target:add(torch.cmul(predictedLabels, self.unlabeledMask))
      self.crit:updateOutput(input, target)
      -- multiplies unlabeled entries by kAnneal.      
      self.crit.output:cmul(self.annealedUnlabeledMask)
   end
   self.output = self.crit.output
   return self.output
end

-- TODO no idea if this is correct.
-- returns/sets the gradient
function PseudoLabelCriterion:updateGradInput(input, target)
   input = input:squeeze()
   if type(target) == 'number' then 
      self.crit:updateGradInput(input, target)
      self.crit.gradInput = self.crit.gradInput * self.annealedUnlabeledMask      
   else
      target = target:squeeze()
      self.crit:updateGradInput(input, target)
      -- TODO This is wrong! only multiply the unlabled gradients
      self.crit.gradInput:cmul(self.annealedUnlabeledMask) -- multiplies unlabeled entries by kAnneal.
   end     
   self.gradInput = self.crit.gradInput
   return self.gradInput
end

end



In [ ]:
-- Single value test: labeled

criterion = nn.ClassNLLCriterion() -- a negative log-likelihood criterion for multi-class classification

-- mask: labeled datum
unlabeledMask = 0
kAnneal = 2
pseudoCriterion = nn.PseudoLabelCriterion(criterion, unlabeledMask, kAnneal)


-- Output thinks the 2nd entry is correct
output = torch.ones(10)*0.1
output[2] = 0.2
output[4] = 0.0
print('output:')
print(output)

-- groundtruth: 3 is correct
gt = 3
error = criterion:forward(output, gt)
gradients = criterion:backward(output, gt)
print('normal error: '..error)
print('normal gradients:')
print(gradients)

pseudoError = pseudoCriterion:forward(output, gt)
pseudoGradients = pseudoCriterion:backward(output, gt)
print('pseudo error: '..pseudoError)
print('pseudo gradients:')
print(pseudoGradients)

-- NOTE: gradients should be equal

In [ ]:
-- Single value test: UNlabeled

criterion = nn.ClassNLLCriterion() -- a negative log-likelihood criterion for multi-class classification

-- mask: unlabeled datum
unlabeledMask = 1
kAnneal = 2
pseudoCriterion = nn.PseudoLabelCriterion(criterion, unlabeledMask, kAnneal)

-- Output thinks the 2nd entry is correct
output = torch.ones(10)*0.1
output[2] = 0.2
output[4] = 0.0
print('output:')
print(output)

-- groundtruth: 3 is correct
gt = 3
error = criterion:forward(output, gt)
gradients = criterion:backward(output, gt)
print('normal error: '..error)
print('normal gradients:')
print(gradients)

pseudoError = pseudoCriterion:forward(output, gt)
pseudoGradients = pseudoCriterion:backward(output, gt)
print('pseudo error: '..pseudoError)
print('pseudo gradients:')
print(pseudoGradients)

-- NOTE: gradients should be different

In [ ]:
-- TODO test with tensors
-- Multivalued test: UNlabeled

criterion = nn.ClassNLLCriterion() -- a negative log-likelihood criterion for multi-class classification

-- mask: three data points, the second is unlabeled.
unlabeledMask = torch.zeros(3)
torch[2] = 1
kAnneal = 2
pseudoCriterion = nn.PseudoLabelCriterion(criterion, unlabeledMask, kAnneal)

-- Output thinks the 2nd entry is correct
output = torch.ones(10)*0.1
output[2] = 0.2
output[4] = 0.0
print('output:')
print(output)



In [ ]:
-- TODO test jacobian

-- parameters
local precision = 1e-5
local jac = nn.Jacobian

-- define inputs and module
local ini = math.random(10,20)
local inj = math.random(10,20)
local ink = math.random(10,20)
local percentage = 0.5
local input = torch.Tensor(ini,inj,ink):zero()
local module = nn.Dropout(percentage)

-- test backprop, with Jacobian
local err = jac.testJacobian(module,input)
print('==> error: ' .. err)
if err<precision then
   print('==> module OK')
else
      print('==> error too large, incorrect implementation')
end



In [22]:
asdf = torch.rand(3,2)
print(asdf)

mask = torch.zeros(3)
mask[2] = 1
print(mask)

--torch.cmul(asdf, mask)

 0.7566  0.5812
 0.1694  0.5796
 0.4557  0.9802
[torch.DoubleTensor of size 3x2]

 0
 1
 0
[torch.DoubleTensor of size 3]

